In [58]:
import os
import pprint
import nibabel as nib

# This is how we will search for the correct file names in the correct directory

def fileList(source):
    matches = []
    for root, dirnames, filenames in os.walk(source):
        for filename in filenames:
            #print(root)
            #print(filenames)
            if ('SUBJ_111') in root:
                # I don't like this hardcoded string,
                # but I don't know how the file labelling is done by medical practitioners
                if 'MR1' in filename and filename.endswith('.img'):
                    matches.append(os.path.join(root,filename))
    return matches

def id_list(source):
    matches = []
    for root, dirnames, filenames in os.walk(source):
        for dirname in dirnames:
            if ('OAS1') and ('MR1') in dirname:
                matches.append(dirname)          
    return matches

patients = fileList('C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\')
# pprint.pprint(patients)
ids = id_list('C:/Users/aditya.i.addepalli/Downloads/OASIS/')
#pprint.pprint(ids)

['C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0001_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0001_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0002_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0002_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0003_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0003_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0004_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0004_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0005_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0005_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0006_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0006_MR1_mpr_n4_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1_0007_MR1\\PROCESSED\\MPRAGE\\SUBJ_111\\OAS1_0007_MR1_mpr_n3_anon_sbj_111.img',
 'C:\\Users\\aditya.i.addepalli\\Downloads\\OASIS\\OAS1

In [36]:
import nibabel as nib
import matplotlib.pyplot as plt
import scipy.misc
import numpy as np
import pandas as pd
import math

#example_file = 'C:/Users/aditya.i.addepalli/Downloads/OASIS/disc1/OAS1_0001_MR1/FSL_SEG/OAS1_0001_MR1_mpr_n4_anon_111_t88_masked_gfc_fseg.img'

# Still have to add for all the files in matches
# This gives me 20+ slices 
# 50,nib.load(example_file).shape[2] - 30,5
def get_slices(example_file):
    count = 0
    loaded_file = nib.load(example_file)
    all_slices =[]
    for i in range(loaded_file.shape[2]):
        #plt.figure()
        #plt.axis('on')
        one_slice = list(np.rot90(scipy.misc.imresize(loaded_file.get_data()[..., i, 0],0.2)))
        #plt.imshow(slices, interpolation = "nearest", cmap=plt.cm.gray)
        #plt.show()
        all_slices.append(one_slice)
    return all_slices

def chunks(l, n):
    # Credit: Ned Batchelder
    # Link: http://stackoverflow.com/questions/312443/how-do-you-split-a-list-into-evenly-sized-chunks
    """Yield successive n-sized chunks from l."""
    for i in range(0, len(l), n):
        yield l[i:i + n]

def mean(l):
    return sum(l) / len(l) 



# for patient in patients[:10]:
#     HM_SLICES = 20   
#     new_slices = []
#     slices = get_slices(patient)
#     chunk_sizes = math.ceil(len(slices) / HM_SLICES)
#     for slice_chunk in chunks(slices, chunk_sizes):
#         slice_chunk = list(map(mean, zip(*slice_chunk)))
#         new_slices.append(slice_chunk)
#     count +=1
    #print(len(slices), len(new_slices))
    




In [63]:
def process_data(labels_df,patient):
    HM_SLICES = 20   
    new_slices = []
    print(patient)
    slices = get_slices(patient)
    chunk_sizes = math.ceil(len(slices) / HM_SLICES)
    for slice_chunk in chunks(slices, chunk_sizes):
        slice_chunk = list(map(mean, zip(*slice_chunk)))
        new_slices.append(slice_chunk)
    print(len(slices), len(new_slices))    
    label = labels_df.get_value(patient.split('\\')[5], 'CDR')
    try:
        if label > 0: label=np.array([0.0,1.0])
        elif label == 0: label=np.array([1.0,0.0])
    except:
        print("unlabelled or wrongly labelled data")
        
    return np.array(new_slices),label   

# img_data,label = process_data(cdr_df,'C:/Users/aditya.i.addepalli/Downloads/OASIS/OAS1_0454_MR1/PROCESSED/MPRAGE/SUBJ_111/OAS1_0454_MR1_mpr_n3_anon_sbj_111.img')
# print(img_data[11][11])
# print(label)

# img_data2,label2 = process_data(cdr_df,'OAS1_001_MR1')
# print(img_data2[11][11])
# print(label2)



In [64]:
labels_file = 'C:/Users/aditya.i.addepalli/Downloads/OASIS/oasis_cross-sectional.csv'
read_labels_df = pd.read_csv(labels_file, index_col=0)
cdr_df = read_labels_df['CDR'].fillna(0)
#cdr_df = read_labels_df['CDR'].fillna('')

    
mri_data = []
for num,patient in enumerate(patients):
    if num % 100 == 0:
        print(num)
    try:
        img_data,label = process_data(cdr_df,patient)
        print(label)
#         print(img_data[10])
        
        mri_data.append([img_data,label])
    # Why isn't this working?    
    except KeyError as e:
       print('This is unlabeled data!')

np.save('final505021.npy', mri_data)


# #print(a.shape)


# # print(len(a[0]))
# # print(len(a))
# # print(loaded_data[0])
# print(len(loaded_data))
# print(len(loaded_data[0]))
# print(loaded_data[0])
# print(len(loaded_data[0][0])) 
# print(len(loaded_data[0][0][0])) 


# new_data =[]
# for i in range(len(loaded_data)):
#     new_data.append([loaded_data[0][0][10],loaded_data[0][1]])
# np.save('MRIdatanew.npy', new_data) 


# print(len(new_data))
# print(len(new_data[0]))
# print(new_data[0])
# print(len(new_data[0][0])) 
# print(len(new_data[0][0][0])) 


# # print(len(a[0][0]))
# # print((a[0][0][0][0]))


0
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0001_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0001_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0002_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0002_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0003_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0003_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0004_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0004_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0005_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0005_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0006_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0006_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0007_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0007_MR1_mpr_n3_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepal

160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0065_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0065_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0066_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0066_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0067_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0067_MR1_mpr_n3_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0068_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0068_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0069_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0069_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0070_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0070_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0071_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0071_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\a

160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0129_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0129_MR1_mpr_n4_fdcanon_sbj_111.4dfp.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0130_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0130_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0131_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0131_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0132_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0132_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0133_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0133_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0134_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0134_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0135_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0135_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:

160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0195_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0195_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0197_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0197_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0198_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0198_MR1_mpr_n4_fdcanon_sbj_111.4dfp.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0199_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0199_MR1_mpr_n3_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0200_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0200_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0201_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0201_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0202_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0202_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:

160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0264_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0264_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0265_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0265_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0266_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0266_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0267_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0267_MR1_mpr_n3_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0268_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0268_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0269_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0269_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0270_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0270_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\a

160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0328_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0328_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0329_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0329_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0330_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0330_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0331_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0331_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
300
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0332_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0332_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0333_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0333_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0335_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0335_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Use

160 20
[ 0.  1.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0392_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0392_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0394_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0394_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0395_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0395_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0396_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0396_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0397_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0397_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0398_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0398_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0399_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0399_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 0.  1.]
C:\Users\a

160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0456_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0456_MR1_mpr_n4_anon_sbj_111.img
160 20
[ 1.  0.]
C:\Users\aditya.i.addepalli\Downloads\OASIS\OAS1_0457_MR1\PROCESSED\MPRAGE\SUBJ_111\OAS1_0457_MR1_mpr_n3_anon_sbj_111.img
160 20
[ 1.  0.]


In [81]:
loaded_data = np.load('final505021.npy')
print((loaded_data[0][1])) 


[ 1.  0.]


In [21]:
loaded_data = np.load('final505021.npy')
for i in range(len(loaded_data[:,1])):
    if np.array_equal(loaded_data[i][1],loaded_data[2][1]):
        print(i+2)

# print(loaded_data[0])
# print(len(loaded_data))
# print(len(loaded_data[0]))
# print(loaded_data[0])
# print(len(loaded_data[0][0])) 
# print(len(loaded_data[0][0][0])) 
# print(len(loaded_data[0][0][0][0])) 


4
15
16
21
22
23
27
30
34
37
39
40
44
49
50
53
57
63
64
70
79
81
89
93
109
113
115
116
117
126
129
134
135
145
148
151
154
156
166
171
172
189
194
200
205
207
214
219
221
223
226
238
242
243
244
247
248
252
260
261
262
264
265
271
273
277
279
280
284
287
288
299
304
308
319
320
329
339
340
346
348
354
356
363
364
366
369
375
380
386
387
391
393
400
401
407
411
412
413
414


In [1]:
import tensorflow as tf
import numpy as np
import math

loaded_data = np.load('C:/Users/aditya.i.addepalli/Videos/three_dim_256_3.npy')
train_data = loaded_data[:-100]
validation_data = loaded_data[-100:]

def required_len_tensor(IMG_SIZE_PX_X,IMG_SIZE_PX_Y,SLICE_COUNT,stride_length,number_of_features):
    return math.ceil(IMG_SIZE_PX_X/stride_length/stride_length) * math.ceil(IMG_SIZE_PX_Y/stride_length/stride_length) *  math.ceil(SLICE_COUNT/stride_length/stride_length) * (number_of_features) 
    
stride_length = 2
number_of_features = 64
IMG_SIZE_PX_X = len(loaded_data[0][0][0])
IMG_SIZE_PX_Y = len(loaded_data[0][0][0][0])
SLICE_COUNT = len(loaded_data[0][0])

req_tensor = required_len_tensor(IMG_SIZE_PX_X,IMG_SIZE_PX_Y,SLICE_COUNT,stride_length,number_of_features)
n_classes = 2
batch_size = 10

x = tf.placeholder('float')
y = tf.placeholder('float')
keep_rate = 0.8
print(req_tensor)

262144


In [2]:
def conv3d(x, W):
    return tf.nn.conv3d(x, W, strides=[1,1,1,1,1], padding='SAME')

def maxpool3d(x):
    #                        size of window         movement of window as you slide about
    return tf.nn.max_pool3d(x, ksize=[1,2,2,2,1], strides=[1,2,2,2,1], padding='SAME')

In [ ]:
def convolutional_neural_network(x):
    #                # 5 x 5 x 5 patches, 1 channel, 32 features to compute.
    weights = {'W_conv1':tf.Variable(tf.random_normal([3,3,3,1,32])),
               #       5 x 5 x 5 patches, 32 channels, 64 features to compute.
               'W_conv2':tf.Variable(tf.random_normal([3,3,3,32,64])),
               #                                  64 features
               'W_fc':tf.Variable(tf.random_normal([req_tensor,128])), 
               'out':tf.Variable(tf.random_normal([128, n_classes]))}

    biases = {'b_conv1':tf.Variable(tf.random_normal([32])),
               'b_conv2':tf.Variable(tf.random_normal([64])),
               'b_fc':tf.Variable(tf.random_normal([128])),
               'out':tf.Variable(tf.random_normal([n_classes]))}

    #                            image X      image Y        image Z
    x = tf.reshape(x, shape=[-1, IMG_SIZE_PX_X, IMG_SIZE_PX_Y, SLICE_COUNT, 1])

    conv1 = tf.nn.relu(conv3d(x, weights['W_conv1']) + biases['b_conv1'])
    conv1 = maxpool3d(conv1)


    conv2 = tf.nn.relu(conv3d(conv1, weights['W_conv2']) + biases['b_conv2'])
    conv2 = maxpool3d(conv2)
    #print(conv2.shape)
    

    fc = tf.reshape(conv2,(-1,req_tensor))
    fc = tf.nn.relu(tf.matmul(fc, weights['W_fc']) + biases['b_fc'])
    fc = tf.nn.dropout(fc, keep_rate)

    output = tf.matmul(fc, weights['out']) + biases['out']

    return output

In [ ]:
#y_pred = []
def train_neural_network(x):
    prediction = convolutional_neural_network(x)
    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(logits=prediction, labels=y) )
#     optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
    
    hm_epochs = 20
    with tf.Session() as sess:
        sess.run(tf.initialize_all_variables())
        
        successful_runs = 0
        total_runs = 0
        
        for epoch in range(hm_epochs):
            epoch_loss = 0
            
            optimizer = tf.train.AdamOptimizer(learning_rate=1e-3).minimize(cost)
            for data in train_data:
                total_runs += 1
                try:
                    X = data[0]
                    
                    Y = data[1]
#                     print(Y)
                    _, c = sess.run([optimizer, cost], feed_dict={x: X, y: Y})
                    epoch_loss += c
                    successful_runs += 1
                except Exception as e:
                    # I am passing for the sake of notebook space, but we are getting 1 shaping issue from one 
                    # input tensor. Not sure why, will have to look into it. Guessing it's
                    # one of the depths that doesn't come to 20.
                    pass
                    #print(str(e))
            
            print('Epoch', epoch+1, 'completed out of',hm_epochs,'loss:',epoch_loss)
            #y_pred.append(tf.argmax(prediction,1).eval()[0])
            #exit(0)
            
            correct = tf.equal(tf.argmax(prediction, 1), tf.argmax(y, 1))
            accuracy = tf.reduce_mean(tf.cast(correct, 'float'))

#             print(' test Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
#             print(correct.eval({x:[i[0] for i in train_data], y:[i[1] for i in train_data]}))
            print('Training Accuracy:',accuracy.eval({x:[i[0] for i in train_data], y:[i[1] for i in train_data]}))
            
            
            
        print('Done. Finishing accuracy:')
        print('Accuracy:',accuracy.eval({x:[i[0] for i in validation_data], y:[i[1] for i in validation_data]}))
        
        print('fitment percent:',successful_runs/total_runs)

# Run this locally:
train_neural_network(x)

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Epoch 1 completed out of 20 loss: 0


In [ ]:
import pandas as pd
labels_file = 'C:/Users/aditya.i.addepalli/Downloads/OASIS/oasis_cross-sectional.csv'
read_labels_df = pd.read_csv(labels_file, index_col=0)
cdr_df = read_labels_df['CDR'].fillna(0)
from sklearn.metrics import confusion_matrix
y_true = []
for i in range(len(cdr_df)-50,len(cdr_df)):
    y_true.append(cdr_df[i])
for i in range(len(y_true)):
    if(y_true[i] > 0):
        y_true[i] = 1
    elif(y_true[i] > 0):
        y_true[i] = 0
#print(len(y_true))

In [ ]:
print(prediction)

In [ ]:
y = tf.placeholder('float')